# 🔄 Resume Training: English-Vietnamese Model

Resume training from existing checkpoint.

---

## 1. ⚙️ Setup

In [ ]:
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard pyyaml

In [ ]:
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
    f.write(f'HF_TOKEN={HF_TOKEN}\n')
print('✓ .env created')

## 2. 🔧 Configure Paths

In [ ]:
import yaml

# Path to your uploaded model dataset
KAGGLE_INPUT_MODEL = '/kaggle/input/en-vi-model'  # CHANGE THIS

CONFIG_FILE = 'config/config.yaml'

with open(CONFIG_FILE, 'r') as f:
    cfg = yaml.safe_load(f)

cfg['paths'] = {
    'data_dir': '/kaggle/working/data',
    'checkpoint_dir': '/kaggle/working/checkpoints',
    'log_dir': '/kaggle/working/logs'
}
cfg['data']['source'] = 'processed'

with open(CONFIG_FILE, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, allow_unicode=True)

print('✓ Config paths updated')

## 3. 📂 Copy Files

In [ ]:
import shutil

!mkdir -p /kaggle/working/data/processed
!mkdir -p /kaggle/working/checkpoints/tokenizers

# Copy files
shutil.copy(f'{KAGGLE_INPUT_MODEL}/train.pt', '/kaggle/working/data/processed/train.pt')
shutil.copy(f'{KAGGLE_INPUT_MODEL}/val.pt', '/kaggle/working/data/processed/val.pt')
shutil.copy(f'{KAGGLE_INPUT_MODEL}/tokenizer_en.model', '/kaggle/working/checkpoints/tokenizers/tokenizer_en.model')
shutil.copy(f'{KAGGLE_INPUT_MODEL}/tokenizer_vi.model', '/kaggle/working/checkpoints/tokenizers/tokenizer_vi.model')
shutil.copy(f'{KAGGLE_INPUT_MODEL}/best_model.pt', '/kaggle/working/checkpoints/best_model.pt')

print('✓ All files copied')

## 4. 🏋️ Resume Training

In [ ]:
import sys
sys.path.insert(0, '.')

from src.train import Trainer
from src.config import load_config

config = load_config('config/config.yaml')
trainer = Trainer(config_path='config/config.yaml')
trainer.setup()

print("\n" + "="*50)
print("🔄 Resuming EN → VI Training")
print("="*50)

resume_path = str(config.paths.checkpoint_dir / 'best_model.pt')
trainer.train(resume_from=resume_path)